In [108]:
import requests
import pandas as pd
import numpy as np

In [109]:
stats=pd.read_excel('statid.xlsx')

In [152]:
url='http://www.nfl.com/ajax/scorestrip?season={}&seasonType={}&week={}'.format(2019,"PRE",1)

In [122]:
import xml.etree.ElementTree as ET

In [153]:
root=ET.fromstring(requests.get(url).text)

In [155]:
games=[]
for i in root[0]:
    games.append(i.attrib['eid'])

In [112]:
gid=2019080860
# for gid in games:
url2='http://www.nfl.com/liveupdate/game-center/{}/{}_gtd.json'.format(gid,gid)

d=requests.get(url2).json()

dfs=[]
for key,val in d[str(gid)]['drives'].items():
    if key =='crntdrv':
        continue
    df=pd.DataFrame(val)[['start','end']].dropna()

    df2=pd.DataFrame([0])
    counter=0
    for i in df.columns:
        counter=0
        for j in df[i]:
            df2[i+' '+df[i].index[counter]]=j
            counter+=1
    df2=df2.drop([0,'start team','end team','end qtr'],axis=1)



    df=pd.DataFrame(val).drop(['plays','start','end'],axis=1).drop_duplicates()
    try:
        df=df.drop('redzone',axis=1)
    except:
        pass

    df=df.reset_index().drop('index',axis=1).reset_index()

    df2=df2.reset_index()

    df=pd.merge(df,df2,on='index').drop('index',axis=1)
    df['driveid']=key
    dfs.append(df)

    df=pd.concat(dfs).fillna(0)

    df['posteam']=df['posteam'].str.pad(3,'right')

    df['end yrdln']=np.where(df['end yrdln']=='',df['start yrdln'],df['end yrdln'])

    df['start yrdln']=np.where(df['posteam']==df['start yrdln'].str[:3],
                               df['start yrdln'].str.split(' ').str[1],
                               (50-df['start yrdln'].str.split(' ').str[1].astype(np.int64))+50)

    df['end yrdln']=df['start yrdln'].astype(np.int64)+df['penyds'].astype(np.int64)+df['ydsgained'].astype(np.int64)
    df['gid']=gid

df

,posteam,qtr,fds,result,penyds,ydsgained,numplays,postime,start qtr,start time,start yrdln,end time,end yrdln,driveid,gid
0,NYJ,1,3,Touchdown,0,75,10,3:07,1,15:00,25,11:53,100,1,2019080860
0,NYG,1,0,Punt,0,4,5,1:50,1,11:53,25,10:03,29,2,2019080860
0,NYJ,1,0,Punt,0,4,4,2:04,1,10:03,22,07:59,26,3,2019080860
0,NYG,1,5,Touchdown,0,79,9,4:23,1,07:59,21,03:36,100,4,2019080860
0,NYJ,1,0,Punt,0,4,5,2:13,1,03:36,16,01:23,20,5,2019080860
0,NYG,2,3,Touchdown,0,81,9,2:42,1,01:23,19,13:41,100,6,2019080860
0,NYJ,2,4,Touchdown,26,49,14,6:17,2,13:41,25,07:24,100,7,2019080860
0,NYG,2,0,Punt,-15,10,7,2:11,2,07:24,34,05:13,29,8,2019080860
0,NYJ,2,1,Punt,-15,22,7,2:30,2,05:13,43,02:43,50,9,2019080860
0,NYG,2,0,Fumble,0,3,1,0:09,2,02:43,13,02:34,16,10,2019080860


In [156]:
gid=2019080860
dfs=[]
for gid in games:
    print gid
    url2='http://www.nfl.com/liveupdate/game-center/{}/{}_gtd.json'.format(gid,gid)

    d=requests.get(url2).json()


    for key,val in d[str(gid)]['drives'].items():
        if key =='crntdrv':
            continue

        for key2,val2 in val['plays'].items():
            for key3,val3 in val2['players'].items():
                df=(pd.DataFrame(val3))
                df['playerid']=key3
                df['play']=key2
                df['drive']=key
                df['description']=val2['desc']
                dfs.append(df)
    df=pd.concat(dfs)


In [157]:
df=pd.merge(df,stats,left_on='statId',right_on='id')

In [158]:
df['play']=df['play'].astype(np.int64)

In [159]:
df['points']=df['yds points'].fillna(0)*df['yards'].fillna(0)+df['existence points'].fillna(0)

In [160]:
# df[(df['playerName']=='S.Darnold')&(df['points']!=0)].sort_values('play').to_clipboard(sep='\t')

In [167]:
df[df['play']==1152].sort_values('play')['description'].iloc[0]

'(11:39) (Shotgun) T.Siemian pass incomplete deep left to J.Bellamy (D.Baker). PENALTY on NYG-D.Baker, Defensive Pass Interference, 26 yards, enforced at NYJ 46 - No Play.'

In [174]:
df[df['description'].str.contains('Defensive Pass Interference')].sort_values('play').to_clipboard(sep='\t')

In [82]:
piv=df.pivot_table(index='playerName',values=['points','yards'],aggfunc=sum)

In [83]:
piv[piv['points']>0]

,points,yards
playerName,,
A.Roberts,3.10,222.0
A.Robinson,9.90,68.0
A.Thielen,21.50,144.0
B.Copeland,1.00,-8.0
B.Powell,3.00,25.0
B.Zylstra,2.80,28.0
C.Herndon,12.20,67.0
D.Bailey,13.00,900.0
D.Burnett,1.40,9.0


In [84]:
df

,clubcode,playerName,sequence,statId,yards,playerid,play,drive,description,id,cat,yds points,existence points,yds,fields,desc,long,points
0,NYJ,P.Nickerson,3,79,0.0,00-0034274,36,1,J.Myers kicks 65 yards from NYJ 35 to MIN 0. H...,79,defense,NaN,NaN,NaN,defense_tkl,Solo tackle,Tackle with no assists. Note: There are no off...,0.0
1,NYJ,J.Adams,3,79,0.0,00-0033872,58,1,(14:55) (Shotgun) L.Murray up the middle to MI...,79,defense,NaN,NaN,NaN,defense_tkl,Solo tackle,Tackle with no assists. Note: There are no off...,0.0
2,NYJ,D.Roberts,2,79,0.0,00-0031628,117,1,(14:29) (Shotgun) M.Boone right guard to NYJ 3...,79,defense,NaN,NaN,NaN,defense_tkl,Solo tackle,Tackle with no assists. Note: There are no off...,0.0
3,NYJ,A.Williamson,7,79,0.0,00-0031263,138,1,(14:04) (Shotgun) K.Cousins pass short middle ...,79,defense,NaN,NaN,NaN,defense_tkl,Solo tackle,Tackle with no assists. Note: There are no off...,0.0
4,MIN,R.Thomas,3,79,0.0,00-0034221,199,2,D.Bailey kicks 63 yards from MIN 35 to NYJ 2. ...,79,defense,NaN,NaN,NaN,defense_tkl,Solo tackle,Tackle with no assists. Note: There are no off...,0.0
5,MIN,J.Johnson,3,79,0.0,00-0033940,244,2,(12:37) B.Powell up the middle to NYJ 24 for 2...,79,defense,NaN,NaN,NaN,defense_tkl,Solo tackle,Tackle with no assists. Note: There are no off...,0.0
6,MIN,D.Hunter,2,79,0.0,00-0031565,274,2,(11:59) B.Powell right guard to NYJ 27 for 3 y...,79,defense,NaN,NaN,NaN,defense_tkl,Solo tackle,Tackle with no assists. Note: There are no off...,0.0
7,MIN,E.Kendricks,7,79,0.0,00-0031554,328,2,(11:12) (Shotgun) S.Darnold pass short right t...,79,defense,NaN,NaN,NaN,defense_tkl,Solo tackle,Tackle with no assists. Note: There are no off...,0.0
8,NYJ,J.Jenkins,2,79,0.0,00-0032970,381,3,(10:27) (Shotgun) L.Murray up the middle to MI...,79,defense,NaN,NaN,NaN,defense_tkl,Solo tackle,Tackle with no assists. Note: There are no off...,0.0
9,NYJ,B.Copeland,3,79,0.0,00-0030010,426,3,(10:07) (No Huddle) S.Diggs left end to MIN 19...,79,defense,NaN,NaN,NaN,defense_tkl,Solo tackle,Tackle with no assists. Note: There are no off...,0.0


In [85]:
df[(df['statId']==83)|(df['statId']==84)]

,clubcode,playerName,sequence,statId,yards,playerid,play,drive,description,id,cat,yds points,existence points,yds,fields,desc,long,points
666,NYJ,B.Copeland,3,83,-13.0,00-0030010,470,3,(9:02) (Shotgun) K.Cousins sacked at MIN 1 for...,83,defense,NaN,1.0,defense_sk_yds,defense_sk,Sack yards (defense),Unassisted sack.,1.0
667,MIN,J.Holmes,2,83,-5.0,00-0034264,4481,33,(1:53) (Shotgun) S.Darnold sacked at MIN 24 fo...,83,defense,NaN,1.0,defense_sk_yds,defense_sk,Sack yards (defense),Unassisted sack.,1.0
722,MIN,D.Hunter,2,84,-1.0,00-0031565,1456,10,(9:59) (Shotgun) S.Darnold sacked at NYJ 30 fo...,84,defense,NaN,0.5,defense_sk_yds,defense_sk,1/2 sack yards (defense),Sack split equally between two players.,0.5
723,MIN,J.Johnson,3,84,-1.0,00-0033940,1456,10,(9:59) (Shotgun) S.Darnold sacked at NYJ 30 fo...,84,defense,NaN,0.5,defense_sk_yds,defense_sk,1/2 sack yards (defense),Sack split equally between two players.,0.5
724,MIN,D.Hunter,3,84,-8.0,00-0031565,3164,25,(:14) S.Darnold sacked at NYJ 7 for -8 yards (...,84,defense,NaN,0.5,defense_sk_yds,defense_sk,1/2 sack yards (defense),Sack split equally between two players.,0.5
725,MIN,J.Kearse,2,84,-8.0,00-0032454,3164,25,(:14) S.Darnold sacked at NYJ 7 for -8 yards (...,84,defense,NaN,0.5,defense_sk_yds,defense_sk,1/2 sack yards (defense),Sack split equally between two players.,0.5
